# Introduction

1. index_col
2. nunique()
3. isin()
4. merging two data sets using left join
5. apply
6. aggregate functions inside apply (max, sum)
7. Group By
    1. ngroups
    2. groups
    3. get_group
    4. applying aggregate functions on groups
    5. filtering groups
    6. Grouping using lambda
    7. Apply + GROUP BY

In [71]:
import numpy as np
import pandas as pd

In [72]:
movies = pd.read_csv('movies.csv')
movies.head()

,Unnamed: 0,id,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day
0,0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday
1,1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,4763,2007,May,Saturday
2,2,43599,245000000,107,880674609,Spectre,6.3,4466,4764,2015,Oct,Monday
3,3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,4765,2012,Jul,Monday
4,5,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,4767,2007,May,Tuesday


**So what kind of questions can we ask from this dataset?**

- **Top 10 most popular movies**, using `popularity`.
- Find the **highest rated movies**, using `vote_average`.
- We can find number of **movies released per year**.
- Find **highest budget movies in a year** using both `budget` and `year`.

**But can we ask more interesting/deeper questions?**

- Do you think we can find the **most productive directors**?
- Which **directors produce high budget films**?
- **Highest and lowest rated movies for every month** in a particular year?

Notice that there's a column **Unnamed: 0** which represents nothing but the index of a row.

**How to get rid of this `Unnamed: 0` col?**

## 1. index_col
==> In pandas, __index_col__ is a parameter used in functions like __read_csv()__ or __read_excel()__ to specify which column should be used as the index (row labels) of the DataFrame that is being created.

==> When you read data from a file into a DataFrame, pandas assigns row labels by default, usually starting from 0.

==>However, sometimes it's useful to use one of the columns from the data itself as the index, especially if that column contains unique identifiers or meaningful labels.

`index_col=0` explicitly states to treat the first column as the index.

The default value is `index_col=None`

In [73]:
movies = pd.read_csv("movies.csv", index_col = 0)
directors = pd.read_csv("directors.csv", index_col = 0)

In [74]:
movies.head(1)

,id,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day
0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday


In [75]:
directors.head(1)

,director_name,id,gender
0,James Cameron,4762,Male


In [76]:
directors.shape

(2349, 3)

In [77]:
movies.shape

(1465, 11)

In [78]:
movies["director_id"].nunique()

199

In [79]:
directors["id"].nunique()

2349

## 2. nunique()
__nunique()__ is a function used to count the number of unique values in a Series or DataFrame. It returns the count of distinct elements in the object.

In [80]:
#We see that there are 199 unique directors in movies dataframe and there are 2349 directors in directors dataframe
#Hence, it makes sense if we perform left join
#However, to perform left join, we need to make sure every director in movies dataframe are in directors dataframe
#For verifying the same, we use isin() function

## 3. isin()

==> The __Series.isin(Series)__ method in pandas can be used to check whether each element in a Series is contained in another Series. It returns a boolean Series of the same length as the original Series, where each element is replaced with a boolean value indicating whether it is contained in the other Series.

==> Only list like objects are allowed to be passed in isin(). If we pass int or float or str, it will result in error

==> List like objects include Series, Index, DataFrame, Numpy Arrays, Lists. For the purpose of isin(), tuples and dictionaries are also considered as list like objects. However, they are not list like objects

In [81]:
movies["director_id"].isin(directors["id"])

0       True
1       True
2       True
3       True
5       True
        ... 
4736    True
4743    True
4748    True
4749    True
4768    True
Name: director_id, Length: 1465, dtype: bool

In [82]:
np.all(movies["director_id"].isin(directors["id"]))

True

## 4. Merge

In Pandas, the merge function is used to combine multiple DataFrame or Series objects based on common columns or indexes. It allows you to perform various types of joins, similar to the SQL join operations. The merge function provides a powerful and flexible way to combine data from different sources.

Here's the basic syntax of the merge function:

___pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None)___

Let's go through the main parameters:

1. __left and right__: These are the two DataFrame or Series objects that you want to merge.

2. __how__: This parameter specifies the type of join to perform. The available options are:

    - __'inner' (default)__: Performs an inner join, which returns the intersection of the two objects based on the join keys.

    - __'outer'__: Performs a full outer join, which returns the union of the two objects based on the join keys, filling in missing values with NaN.

    - __'left'__: Performs a left outer join, which returns all rows from the left object and only the matching rows from the right object, filling in missing values with NaN.

    - __'right'__: Performs a right outer join, which returns all rows from the right object and only the matching rows from the left object, filling in missing values with NaN.

3. __on__: This parameter specifies the column(s) to use as the join key(s). If on is not provided, and both left_on and right_on are not specified, the merge will attempt to use the overlapping column names as the join keys.

4. __left_on and right_on__: These parameters allow you to specify the column(s) to use as the join key(s) for the left and right objects, respectively. This is useful when the column names are different in the two objects.

In [83]:
import pandas as pd

# Create two DataFrames
employees = pd.DataFrame({'emp_id': [1, 2, 3], 'name': ['John', 'Jane', 'Bob']})
departments = pd.DataFrame({'emp_id': [1, 2, 4], 'dept': ['IT', 'HR', 'Sales']})

# Perform an inner join
merged = pd.merge(employees, departments, on='emp_id', how='inner')
print(merged)

   emp_id  name dept
0       1  John   IT
1       2  Jane   HR


### Merging `movies` & `directors` datasets

**How can we know the details about the Director of a particular movie?**

- We will have to merge these two datasets.

**So on which column we should merge?**

We will use the **ID** columns (representing unique directors) in both the datasets.

If you observe,

- `director_id` of movies are taken from `id` of directors.
- Thus, we can merge our dataframes based on these two columns as **keys**.

Before that, let's first check the number of unique directors in our `movies` dataset.

**How do we get the number of unique directors in `movies`?**


In [84]:
movies['director_id'].nunique()

199

In [85]:
directors['id'].nunique()

2349

**Summary:**

- `movies` dataset: 1465 rows, but only 199 unique directors
- `directors` dataset: 2349 unique directors (equal to the no. of rows)

**What can we infer from this?**

- The directors in `movies` data is a subset of directors in `directors` data.

**How can we check if all `director_id` values are present in `id`?**

In [86]:
movies['director_id'].isin(directors['id'])

0       True
1       True
2       True
3       True
5       True
        ... 
4736    True
4743    True
4748    True
4749    True
4768    True
Name: director_id, Length: 1465, dtype: bool

The `isin()` method checks if a column contains the specified value(s).

**How is `isin` different from Python's `in`?**

- `in` works for **one element** at a time.
- `isin` does this for **all the values** in the column.

If you notice,

- This is like a **boolean mask**.
- It returns a dataframe similar to the original one.
- For rows with values of `director_id` present in `id`, it returns True, else False.

**How can we check if there's any False here?**

In [87]:
np.all(movies['director_id'].isin(directors['id']))

True

Let's finally merge the two dataframes.

Do we need to keep **all the rows for movies**? Yes!

Do we need to keep **all the rows of directors**? No.

- Only the ones for which we have a corresponding row in `movies`.

**So which `join` type do you think we should apply here?**

- `LEFT` Join

In [88]:
data = movies.merge(directors, how='left', left_on='director_id',right_on='id')
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day,director_name,id_y,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday,James Cameron,4762,Male
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,4763,2007,May,Saturday,Gore Verbinski,4763,Male
2,43599,245000000,107,880674609,Spectre,6.3,4466,4764,2015,Oct,Monday,Sam Mendes,4764,Male
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,4765,2012,Jul,Monday,Christopher Nolan,4765,Male
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,4767,2007,May,Tuesday,Sam Raimi,4767,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,4809,1978,May,Monday,Martin Scorsese,4809,Male
1461,48370,27000,19,3151130,Clerks,7.4,755,5369,1994,Sep,Tuesday,Kevin Smith,5369,Male
1462,48375,0,7,0,Rampage,6.0,131,5148,2009,Aug,Friday,Uwe Boll,5148,Male
1463,48376,0,3,0,Slacker,6.4,77,5535,1990,Jul,Friday,Richard Linklater,5535,Male


Notice the two strange id columns - `id_x` and `id_y`.

**What do you think these newly created columns are?**

Since the columns with name `id` are present in both the dataframes,
- `id_x` represents **id values from movie df**
- `id_y` represents **id values from directors df**

**Do you think any column is redundant here and can be dropped?**

- `id_y` is redundant as it is the same as `director_id`
- But we don't require the `director_id` any further.

So we can simply drop these features -

In [89]:
data.drop(['director_id','id_y'], axis=1, inplace=True)
data.head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male


### Post-read

- [IMDB data exploration](https://colab.research.google.com/drive/1yrfHSQYUMxxLKGUG-gCPf-R232BuimiR?usp=sharing)

From here, we have the opportunity to delve into various aspects of the data, such as:

- Converting the revenue values into Millions of USD.
- Identifying the Top 5 most popular movies.

... and so on.

This task is for you to explore the data on your own.

Additionally, we've provided a notebook (accessible as **post-lecture content**) where this analysis has been conducted.

### Quiz-1

# Question
How to filter those records where movies are released either in the year 2015, 2016 or 2012 from the `movies` dataset?

# Choices
- [x] `df['year'].isin([2015, 2016, 2012])`
- [ ] `df['year'].in([2015, 2016, 2012])`
- [ ] `df['year']==([2015, 2016, 2012])`
- [ ] None of the above

## 5. Apply

The __Series.apply(function)__ is a method used to apply a function to each element of a Series. The function can be a built-in function, a lambda function, or any custom function you define.

In Pandas, the apply method is a powerful tool that allows you to apply a function along the rows or columns of a DataFrame or Series object. It provides a flexible way to perform element-wise transformations, calculations, or any custom operations on your data.

The default axis is `axis=0`. This means the function is applied to each column by default.

Here's the basic syntax of the apply method:

__DataFrame.apply(func, axis='columns')__

__Series.apply(func)__

In [90]:
import pandas as pd #Apply a function to each element of a Series

s = pd.Series([1, 2, 3, 4])
squared = s.apply(lambda x: x**2)
print(squared)

0     1
1     4
2     9
3    16
dtype: int64


In [91]:
import pandas as pd #Apply a function row-wise on a DataFrame

df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
row_sum = df.apply(lambda row: row.sum(), axis=1)
print(row_sum)

0    12
1    15
2    18
dtype: int64


In [92]:
import pandas as pd #Apply a custom function column-wise on a DataFrame

df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})

def square_and_add_one(x):
    return (x ** 2) + 1

squared_cols = df.apply(square_and_add_one)
print(squared_cols)

    A   B   C
0   2  17  50
1   5  26  65
2  10  37  82


The apply method is incredibly versatile and can be used for various data manipulation tasks, such as:

- Applying mathematical or statistical functions to data
- Applying string operations or regular expressions
- Implementing custom logic or transformations on data
- Filtering or cleaning data based on specific conditions
- Combining or aggregating data in different ways



### `apply()`

- It is used apply a function along an axis of the DataFrame/Series.

Say we want to convert the data in `Gender` column into numerical format.

Basically,
- 0 for Male
- 1 for Female

**How can we encode the values in the `Gender` column?**

Let's first write a function to do it for a single value.

In [93]:
#we are apply the function columns wise
def encode(data):
    if data == "Male":
        return 0
    else:
        return 1

**Now how can we apply this function to the whole column?**

In [94]:
data['gender'] = data['gender'].apply(encode)
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0
1461,48370,27000,19,3151130,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0
1462,48375,0,7,0,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0
1463,48376,0,3,0,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0


#To apply row wise use the following function

__def encode_gender1(date):__
    
    if data["gender"] = "Male":
    
        return 0
    
    else:
        return 1


__data["gender"] = data.apply(encode_gender1, axis = 1)__

Notice how this is similar to using `Vectorization` in Numpy.

**How to apply a function on multiple columns?**

Let's say we want to find the sum of `revenue` and `budget` per movie?

In [95]:
data[['revenue', 'budget']].apply(np.sum)

revenue    209866997305
budget      70353617179
dtype: int64

In [96]:
data[['revenue', 'budget']].apply(np.sum, axis=1)

0       3024965087
1       1261000000
2       1125674609
3       1334939099
4       1148871626
           ...    
1460        321952
1461       3178130
1462             0
1463             0
1464       2260920
Length: 1465, dtype: int64

By setting the `axis=1`, every row of `revenue` was added to same row of `budget`.

**What does this `axis` mean in apply?**
- `axis=0` $\rightarrow$ It will apply to **each column**
- `axis=1` $\rightarrow$ It will apply to **each row**
  
Note that **by default, axis=0**.

Why are we writing ___data[['revenue']].apply(np.max)___ and not ___data['revenue'].apply(np.max)___

<br>

__data[['revenue']].apply(np.max)__:

==> data[['revenue']] selects the revenue column but returns a DataFrame with one column, not a Series.

==> apply(np.max) on a DataFrame __applies the function along a specified axis (default is axis=0, i.e., along columns). This means it applies np.max to each column individually and returns the maximum value for each column.__

==> The result is a Series where each element corresponds to the maximum value of the respective column in the DataFrame.

<br>

__data['revenue'].apply(np.max)__:

==> data['revenue'] selects the revenue column and returns a Series.

==> apply(np.max) on a Series __applies the function element-wise. However, since np.max is a reduction operation (it computes the maximum of an array), it doesn't make sense to apply it element-wise to each individual value in the Series.__

==> Instead, you should directly use np.max(data['revenue']) or data['revenue'].max(), which computes the maximum value of the entire Series.

**Similarly, how can I find the `profit` per movie (revenue-budget)?**

In [97]:
# We define a function to calculate profit
#We are applying the function row wise

def prof(x):
      return x['revenue']-x['budget']

data['profit'] = data.apply(prof, axis = 1)
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550965087
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661000000
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635674609
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834939099
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632871626
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0,321952
1461,48370,27000,19,3151130,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0,3124130
1462,48375,0,7,0,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0,0
1463,48376,0,3,0,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0,0


___def___ prof(x):
    
    return x["revenue"] - x["budget"]

The function prof(x) calculates the profit, which is typically the revenue minus the expenses or budget. It assumes that the input x is a __dictionary-like object or a pandas Series where the keys or column names are "revenue" and "budget"__, and the corresponding values are numerical, representing the revenue and budget, respectively.

## 7. Grouping / Group By

### What is Grouping?

In simple terms, we could understood it through - Split, Apply, Combine

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781491912126/files/assets/pyds_03in01.png" height="350" width="700"/>

1. **Split**: Breaking up and grouping a DataFrame depending on the value of the specified key.

2. **Apply**: Computing some function, usually an aggregate, transformation, or filtering, within the individual groups.

3. **Combine**: Merging the results of these operations into an output array.

==> The __groupby()__ function in pandas is used to split the DataFrame into groups based on some criteria. When you use groupby() with a Series, it groups the DataFrame by the unique values in that Series.

    Syntax: DataFrame.groupby("Column_Name")
    
==> The __DataFrame.groupby("Column_Name").ngroups__ attribute in pandas is used to return the number of groups formed after applying the groupby() function

==> The __DataFrame.groupby("Column_Name").groups__ attribute in pandas is used to return a dictionary where the keys are the unique group names (in this case, director names) and the values are the indices of the rows that belong to each group

In [98]:
data.groupby('director_name')

Notice,
- It's a **DataFrameGroupBy** type object
- **NOT a DataFrame** type object

**What's the number of groups our data is divided into?**

In [99]:
data.groupby('director_name').ngroups

199

Based on this grouping, we can find which keys belong to which group.

In [100]:
data.groupby('director_name').groups

{'Adam McKay': [176, 323, 366, 505, 839, 916], 'Adam Shankman': [265, 300, 350, 404, 458, 843, 999, 1231], 'Alejandro González Iñárritu': [106, 749, 1015, 1034, 1077, 1405], 'Alex Proyas': [95, 159, 514, 671, 873], 'Alexander Payne': [793, 1006, 1101, 1211, 1281], 'Andrew Adamson': [11, 43, 328, 501, 947], 'Andrew Niccol': [533, 603, 701, 722, 1439], 'Andrzej Bartkowiak': [349, 549, 754, 911, 924], 'Andy Fickman': [517, 681, 909, 926, 973, 1023], 'Andy Tennant': [314, 320, 464, 593, 676, 885], 'Ang Lee': [99, 134, 748, 840, 1089, 1110, 1132, 1184], 'Anne Fletcher': [610, 650, 736, 789, 1206], 'Antoine Fuqua': [310, 338, 424, 467, 576, 808, 818, 1105], 'Atom Egoyan': [946, 1128, 1164, 1194, 1347, 1416], 'Barry Levinson': [313, 319, 471, 594, 878, 898, 1013, 1037, 1082, 1143, 1185, 1345, 1378], 'Barry Sonnenfeld': [13, 48, 90, 205, 591, 778, 783], 'Ben Stiller': [209, 212, 547, 562, 850], 'Bill Condon': [102, 307, 902, 1233, 1381], 'Bobby Farrelly': [352, 356, 481, 498, 624, 630, 654, 80

Let's group our data director-wise.

**What if we want to extract data of a particular group from this list?**

In [101]:
data.groupby('director_name').get_group('Alexander Payne')

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
793,45163,30000000,19,105834556,About Schmidt,6.7,362,2002,Dec,Friday,Alexander Payne,1,75834556
1006,45699,20000000,40,177243185,The Descendants,6.7,934,2011,Sep,Friday,Alexander Payne,1,157243185
1101,46004,16000000,23,109502303,Sideways,6.9,478,2004,Oct,Friday,Alexander Payne,1,93502303
1211,46446,12000000,29,17654912,Nebraska,7.4,636,2013,Sep,Saturday,Alexander Payne,1,5654912
1281,46813,0,13,0,Election,6.7,270,1999,Apr,Friday,Alexander Payne,1,0


**How can we find the count of movies by each director?**

In [102]:
data.groupby('director_name')['title'].count()

director_name
Adam McKay                      6
Adam Shankman                   8
Alejandro González Iñárritu     6
Alex Proyas                     5
Alexander Payne                 5
                               ..
Wes Craven                     10
Wolfgang Petersen               7
Woody Allen                    18
Zack Snyder                     7
Zhang Yimou                     6
Name: title, Length: 199, dtype: int64

### Group based Aggregation

We use different aggregate functions like `mean`, `sum`, `min`, `max`, `count` etc. on columns while grouping.

**How to find multiple aggregates of any feature?**

Finding the very first year and the latest year a director released a movie i.e basically the **min** & **max** of the `year` column, grouped by `director_name`.

In [103]:
data.groupby(['director_name'])["year"].aggregate(['min', 'max'])

,min,max
director_name,,
Adam McKay,2004,2015
Adam Shankman,2001,2012
Alejandro González Iñárritu,2000,2015
Alex Proyas,1994,2016
Alexander Payne,1999,2013
...,...,...
Wes Craven,1984,2011
Wolfgang Petersen,1981,2006
Woody Allen,1977,2013


**Note:** We can also use `.agg` instead of `.aggregate` (both are same)

the __aggregate()__ function, also known as __agg()__, is used to compute multiple summary statistics or custom functions simultaneously across one or more columns of a DataFrame.

You can use aggregate() with built-in functions like sum, mean, min, max, etc., or with custom functions that you define.

### Group based Filtering

Group based filtering allows us to filter rows from each group by using conditional statements on each group rather than the whole dataframe.

**How to find the details of the movies by high budget directors?**

- Lets assume, high budget director -> any director with **atleast one movie with budget >100M**.

We can solve this by two ways:
   - Using applying normal conditional filter method
   - Using .filter() method

#### Using applying normal conditional filter method
1. We can **group** the data by director and use `max` of the budget as aggregator.

In [104]:
data_dir_budget = data.groupby("director_name")["budget"].max().reset_index()
data_dir_budget.head()

,director_name,budget
0,Adam McKay,100000000
1,Adam Shankman,80000000
2,Alejandro González Iñárritu,135000000
3,Alex Proyas,140000000
4,Alexander Payne,30000000


2. We can **filter** out the director names with **max budget >100M**.

In [105]:
names = data_dir_budget.loc[data_dir_budget["budget"] >= 100, "director_name"]

3. Finally, we can filter out the details of the movies by these directors.

In [106]:
data.loc[data['director_name'].isin(names)]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550965087
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661000000
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635674609
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834939099
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632871626
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0,321952
1461,48370,27000,19,3151130,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0,3124130
1462,48375,0,7,0,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0,0
1463,48376,0,3,0,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0,0


#### Using .filter() method

**Can we filter groups in a single go using Lambda functions?** Yes!

In [107]:
data.groupby('director_name').filter(lambda x: x["budget"].max() >= 100)

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550965087
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661000000
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635674609
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834939099
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632871626
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0,321952
1461,48370,27000,19,3151130,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0,3124130
1462,48375,0,7,0,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0,0
1463,48376,0,3,0,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0,0


Notice what's happening here?

- We first group data by director and then use `groupby().filter` function.
- **Groups are filtered if they do not satisfy the boolean criterion** specified by the function.
- This is called **Group Based Filtering**.

**Note:**
- We are filtering the **groups** here and **not the rows**.
- The result is **not a groupby object** but regular **Pandas DataFrame** with the **filtered groups eliminated**.

### Group based Apply

- applying a function on grouped objects

**What if we want to do the transformation of a column using some column's agrregate**

Let's say, we want to filter the risky movies whose budget was even higher than the average revenue of the director from his other movies.

We can subtract the average `revenue` of a director from `budget` column, for each director.

In [108]:
def func(x):
  # returns whether a movie is risky or not
      x["risky"] = x["budget"] - x["revenue"].mean() >= 0
      return x

data_risky = data.groupby("director_name", group_keys=False).apply(func)
data_risky

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit,risky
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550965087,False
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661000000,False
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635674609,False
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834939099,False
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632871626,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0,321952,False
1461,48370,27000,19,3151130,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0,3124130,False
1462,48375,0,7,0,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0,0,False
1463,48376,0,3,0,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0,0,False


**Note:**
- Setting `group_keys=True`, keeps the group key in the returned dataset.
- This will be default in future versions of Pandas.
- Keep it as False if want the normal behaviour.

**What did we do here?**

- Defined a custom function.
- Grouped data according to `director_name`.
- Subtracted the mean of `budget` from `revenue`.
- Used apply with the custom function on the grouped data.

Now let's see if there are any risky movies -

In [109]:
data_risky.loc[data_risky["risky"]]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit,risky
7,43608,200000000,107,586090727,Quantum of Solace,6.1,2965,2008,Oct,Thursday,Marc Forster,0,386090727,True
12,43614,380000000,135,1045713802,Pirates of the Caribbean: On Stranger Tides,6.4,4948,2011,May,Saturday,Rob Marshall,0,665713802,True
15,43618,200000000,37,310669540,Robin Hood,6.2,1398,2010,May,Wednesday,Ridley Scott,0,110669540,True
20,43624,209000000,64,303025485,Battleship,5.5,2114,2012,Apr,Wednesday,Peter Berg,0,94025485,True
24,43630,210000000,3,459359555,X-Men: The Last Stand,6.3,3525,2006,May,Wednesday,Brett Ratner,0,249359555,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1347,47224,5000000,7,3263585,The Sweet Hereafter,6.8,103,1997,May,Wednesday,Atom Egoyan,0,-1736415,True
1349,47229,5000000,3,4842699,90 Minutes in Heaven,5.4,40,2015,Sep,Friday,Michael Polish,0,-157301,True
1351,47233,5000000,6,0,Light Sleeper,5.7,15,1992,Aug,Friday,Paul Schrader,1,-5000000,True
1356,47263,15000000,10,0,Dying of the Light,4.5,118,2014,Dec,Thursday,Paul Schrader,1,-15000000,True


## What is a Group Key??

The group_keys parameter controls whether the keys of the group are included in the index of the result. By default, group_keys is set to True, which means the keys are included. Setting group_keys=False excludes these keys from the resulting DataFrame.

In [110]:
def func(x):
  # returns whether a movie is risky or not
      x["risky"] = x["budget"] - x["revenue"].mean() >= 0
      return x

data_risky = data.groupby("director_name", group_keys=False).apply(func)
data_risky.head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit,risky
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550965087,False
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661000000,False
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635674609,False
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834939099,False
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632871626,False


In [111]:
data_risky = data.groupby("director_name", group_keys=True).apply(func)
data_risky.head()

#director_name = Adam McKay in column 0 is the group_key

id_x     budget  popularity    revenue  \
director_name                                                
Adam McKay    176  43882  100000000          24  170432927   
              323  44151   72500000          12  162966177   
              366  44236   65000000          22  128107642   
              505  44503   50000000          38  173649015   
              839  45301   28000000          57  133346506   

                                                         title  vote_average  \
director_name                                                                  
Adam McKay    176                               The Other Guys           6.1   
              323  Talladega Nights: The Ballad of Ricky Bobby           6.2   
              366                                Step Brothers           6.5   
              505            Anchorman 2: The Legend Continues           6.0   
              839                                The Big Short           7.3   

                   vote_count  year month        day director_name  gender  \
director_name                                                                
Adam McKay    176        1383  2010   Aug     Friday    Adam McKay       0   
              323         491  2006   Aug     Friday    Adam McKay       0   
              366        1062  2008   Jul     Friday    Adam McKay       0   
              505         923  2013   Dec  Wednesday    Adam McKay       0   
              839        2607  2015   Dec     Friday    Adam McKay       0   

                      profit  risky  
director_name                        
Adam McKay    176   70432927  False  
              323   90466177  False  
              366   63107642  False  
              505  123649015  False  
              839  105346506  False